<a href="https://colab.research.google.com/github/sumanapalle065/LC25000-Hybrid-Architecture-using-DL/blob/main/LC25000_Hybrid_Arc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
# Import necessary libraries

import tensorflow as lc2k_tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as lc2k_plt
import os
